In [110]:

#for a buy we want the low of the mid candle to be above the EMA 200, the RSI to be above 50, and the candle should be green and engulfing the prior candle


In [111]:
import sys
sys.path.append("../")


In [112]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicators import RSI
from technicals.patterns import apply_patterns
from exploration.plotting import CandlePlot

In [113]:
df_raw = pd.read_pickle("../data/GBP_JPY_H1.pkl")

In [114]:
df_an = df_raw.copy()
df_an.reset_index(drop=True, inplace=True) #remember to always do this before calcing stuff if you are taking from the end, as our RSI code cares about indices and is expecting them to start from 1

In [115]:
df_an =RSI(df_an)

In [116]:
df_an = apply_patterns(df_an)

In [117]:
df_an['EMA_200'] = df_an.mid_c.ewm(span=200, min_periods=200).mean() #finding the 200 period exponential moving average

In [118]:
our_cols = ['time', 'mid_o', 'mid_l','mid_h', 'mid_c', 'ask_c', 'bid_c','ENGULFING','direction','EMA_200', 'RSI_14']

In [119]:
df_slim = df_an[our_cols].copy()
df_slim.dropna(inplace=True) #as first 14 cant have an RSI as its a 14 average one so get rid of these
df_an.reset_index(drop=True, inplace=True)

In [120]:
BUY = 1
SELL = -1
NONE = 0
RSI_LIMIT = 50.0

def apply_signal(row):   #coding in whether any of the candles fit the stratergy
    if row.ENGULFING == True:
        if row.direction == BUY and row.mid_l > row.EMA_200:
            if row.RSI_14 > RSI_LIMIT:
                return BUY
        if row.direction == SELL and row.mid_h < row.EMA_200:
            if row.RSI_14 < RSI_LIMIT:
                return SELL
    return NONE

In [121]:
df_slim['SIGNAL'] = df_slim.apply(apply_signal, axis=1)

In [122]:
df_slim['SIGNAL'].value_counts()

SIGNAL
 0    41762
 1     2233
-1     1932
Name: count, dtype: int64

In [123]:
LOSS_FACTOR = -1.0
PROFIT_FACTOR = 1.5

def apply_take_profit(row):
    if row.SIGNAL != NONE:
        return (row.mid_c-row.mid_o)*PROFIT_FACTOR + row.mid_c
    else:
        return 0.0

def apply_stop_loss(row):
    if row.SIGNAL != NONE:
        return row.mid_o
    else:
        return 0.0

In [124]:
df_slim['TP'] = df_slim.apply(apply_take_profit, axis=1)
df_slim['SL'] = df_slim.apply(apply_stop_loss, axis=1)

In [125]:
df_plot = df_slim.iloc[0:40]  #only look at first 40 candles
cp = CandlePlot(df_plot, candles=True)

trades = cp.df_plot[df_plot.SIGNAL != NONE] #taking only our candles which are trades

markers = ['mid_c', 'TP', 'SL']
marker_colors = ['#0000FF', '#00FF00', '#FF0000']

for i in range(3):
    cp.fig.add_trace(go.Scatter(x = trades.sTime, y=trades[markers[i]], mode='markers', marker=dict(color=marker_colors[i],size=12)))

cp.show_plot(line_traces=['EMA_200'], sec_traces=['RSI_14'])

In [126]:
class Trade:

    def __init__(self,row):
        self.running = True
        self.start_index = row.name
        self.start_price = row.mid_c
        self.end_price = row.mid_c
        self.SIGNAL = row.SIGNAL
        self.TP = row.TP
        self.SL = row.SL
        self.result = 0.0
        self.end_time = row.time
        self.start_time = row.time
        self.duration = 0

    def close_trade(self,row, result, trigger_price):
        self.running = False
        self.result = result
        self.end_time = row.time
        self.trigger_price = trigger_price

    def update(self,row):
        self.duration += 1
        if self.SIGNAL == BUY:
            if row.mid_h >= self.TP:  #as this is first we are being optimistic for all the cases where the candle that triggers the take profit also covers the stop loss, due to lack of granularity
                self.close_trade(row, PROFIT_FACTOR, row.mid_h)
            elif row.mid_l <= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_l)

        if self.SIGNAL == SELL:
            if row.mid_l <= self.TP:  
                self.close_trade(row, PROFIT_FACTOR, row.mid_l)
            elif row.mid_h >= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_h)




In [127]:
open_trades = []
closed_trades = []

for index, row in df_slim.iterrows(): #iterrows changes the format of the dataframe into a table where you can get the values by doing row.mid_h where row corresponds to the row in the original dataframe you want, note it has lower performance as it makes a series for each row
    for ot in open_trades: #itterate through each still open trade and check that if the current candle we are on leads to a stop loss or take profit, if so the trade is then closed
        ot.update(row)
        if ot.running == False:
            closed_trades.append(ot)
    open_trades = [x for x in open_trades if x.running == True] 

    if row.SIGNAL != NONE: #check if current row is a trade
        open_trades.append(Trade(row))


In [128]:
df_results = pd.DataFrame.from_dict([vars(x) for x in closed_trades])

In [129]:
df_results.sort_values(by='start_index', inplace=True)

In [130]:
df_m5 = pd.read_pickle('../data/GBP_JPY_M5.pkl')

In [131]:
df_raw.time.min()


Timestamp('2016-01-03 22:00:00+0000', tz='tzutc()')

In [132]:
df_m5.time.min() #checking they both have the same start time

Timestamp('2016-01-03 22:00:00+0000', tz='tzutc()')

In [133]:
df_m5.time.max()

Timestamp('2023-05-31 23:55:00+0000', tz='tzutc()')

In [134]:
df_raw.time.max() #checking if they finish on the same time

Timestamp('2023-05-31 23:00:00+0000', tz='tzutc()')

In [135]:
from dateutil import parser

In [136]:
time_min = parser.parse('2021-12-15T10:00:00Z') #make sure its a weekday
time_max = parser.parse('2021-12-15T11:00:00Z')
df_m5_s = df_m5[(df_m5.time>=time_min)&(df_m5.time<=time_max)]
df_raw_s = df_raw[(df_raw.time>=time_min)&(df_raw.time<=time_max)]

In [137]:
df_m5_s

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
443920,2021-12-15 10:00:00+00:00,218,150.826,150.826,150.758,150.806,150.813,150.813,150.745,150.793,150.838,150.838,150.771,150.819
443921,2021-12-15 10:05:00+00:00,152,150.808,150.838,150.772,150.772,150.796,150.826,150.759,150.759,150.821,150.851,150.784,150.784
443922,2021-12-15 10:10:00+00:00,207,150.774,150.882,150.760,150.874,150.761,150.868,150.748,150.862,150.787,150.895,150.772,150.885
443923,2021-12-15 10:15:00+00:00,143,150.876,150.930,150.863,150.890,150.863,150.916,150.850,150.877,150.888,150.943,150.876,150.902
443924,2021-12-15 10:20:00+00:00,197,150.892,150.898,150.814,150.868,150.879,150.886,150.801,150.856,150.904,150.911,150.826,150.881
443925,2021-12-15 10:25:00+00:00,159,150.866,150.896,150.838,150.868,150.855,150.883,150.825,150.855,150.878,150.909,150.850,150.881
443926,2021-12-15 10:30:00+00:00,106,150.870,150.912,150.846,150.890,150.857,150.900,150.834,150.879,150.884,150.924,150.859,150.902
443927,2021-12-15 10:35:00+00:00,80,150.887,150.891,150.834,150.834,150.874,150.877,150.822,150.822,150.900,150.905,150.846,150.846
443928,2021-12-15 10:40:00+00:00,170,150.836,150.899,150.822,150.855,150.824,150.886,150.808,150.843,150.848,150.912,150.836,150.867
443929,2021-12-15 10:45:00+00:00,148,150.852,150.896,150.824,150.890,150.840,150.884,150.813,150.877,150.865,150.908,150.836,150.903


In [138]:
df_raw_s #notice how the corresponding m5 candle with the same mid_c is the 10:55, hence if you are checking on the m5 candles for greater granularity check from 10:55 onwards and not 10 onwards

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
37020,2021-12-15 10:00:00+00:00,1818,150.826,150.948,150.758,150.948,150.813,150.936,150.745,150.935,150.838,150.960,150.771,150.960
37021,2021-12-15 11:00:00+00:00,1842,150.950,150.974,150.784,150.916,150.938,150.961,150.770,150.904,150.961,150.987,150.796,150.927


In [139]:
df_m5_slim = df_m5[['time', 'mid_h','mid_l']].copy()

In [140]:
df_signals = df_slim[df_slim.SIGNAL != NONE].copy()

In [141]:
df_signals['m5_start'] = [x + dt.timedelta(hours=1) for x in df_signals.time] #define the m5 candle start as 1 hour later as from what we saw above from when the prices line up

In [142]:
df_signals['start_index_h1'] = df_signals.index

In [143]:
df_signals.drop(['time', 'mid_o', 'mid_h', 'direction', 'mid_l', 'ask_c', 'bid_c', 'ENGULFING', 'EMA_200', 'RSI_14'], axis=1, inplace=True)

In [144]:
df_signals.head()

,mid_c,SIGNAL,TP,SL,m5_start,start_index_h1
202,169.624,-1,168.8740,170.124,2016-01-14 09:00:00+00:00,202
224,169.552,-1,169.2580,169.748,2016-01-15 07:00:00+00:00,224
228,168.300,-1,167.5845,168.777,2016-01-15 11:00:00+00:00,228
230,168.248,-1,167.9150,168.470,2016-01-15 13:00:00+00:00,230
233,167.240,-1,166.3670,167.822,2016-01-15 16:00:00+00:00,233


In [145]:
df_signals.rename(columns={
    'mid_c' : 'start_price',
    'm5_start' : 'time'
}, inplace=True)

In [146]:
df_m5_slim.head()

,time,mid_h,mid_l
0,2016-01-03 22:00:00+00:00,177.178,177.130
1,2016-01-03 22:05:00+00:00,177.302,177.154
2,2016-01-03 22:10:00+00:00,177.292,177.186
3,2016-01-03 22:15:00+00:00,177.274,177.205
4,2016-01-03 22:20:00+00:00,177.295,177.196


In [147]:
merged = pd.merge(left=df_m5_slim, right=df_signals, on='time', how='left') 
merged.fillna(0,inplace=True) # makes all of the NAs into 0

In [148]:
merged[merged.SIGNAL.isna()==False].head()#gives us the first 5 rows where Signal isnt equal to false
merged.SIGNAL = merged.SIGNAL.astype(int)
merged.start_index_h1 = merged.start_index_h1.astype(int)

In [149]:
merged.head()

,time,mid_h,mid_l,start_price,SIGNAL,TP,SL,start_index_h1
0,2016-01-03 22:00:00+00:00,177.178,177.130,0.0,0,0.0,0.0,0
1,2016-01-03 22:05:00+00:00,177.302,177.154,0.0,0,0.0,0.0,0
2,2016-01-03 22:10:00+00:00,177.292,177.186,0.0,0,0.0,0.0,0
3,2016-01-03 22:15:00+00:00,177.274,177.205,0.0,0,0.0,0.0,0
4,2016-01-03 22:20:00+00:00,177.295,177.196,0.0,0,0.0,0.0,0


In [154]:
class TradeM5:

    def __init__(self,row):
        self.running = True
        self.start_index_m5 = row.name
        self.start_index_h1 = row.start_index_h1
        self.start_price = row.start_price
        self.trigger_price = row.start_price
        self.SIGNAL = row.SIGNAL
        self.TP = row.TP
        self.SL = row.SL
        self.result = 0.0
        self.end_time = row.time
        self.start_time = row.time
        self.duration = 1

    def close_trade(self,row, result, trigger_price):
        self.running = False
        self.result = result
        self.end_time = row.time
        self.trigger_price = trigger_price

    def update(self,row):
        self.duration += 1
        if self.SIGNAL == BUY:
            if row.mid_h >= self.TP:  #as this is first we are being optimistic for all the cases where the candle that triggers the take profit also covers the stop loss, due to lack of granularity
                self.close_trade(row, PROFIT_FACTOR, row.mid_h)
            elif row.mid_l <= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_l)

        if self.SIGNAL == SELL:
            if row.mid_l <= self.TP:  
                self.close_trade(row, PROFIT_FACTOR, row.mid_l)
            elif row.mid_h >= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_h)

In [155]:
open_trades_m5 = []
closed_trades_m5 = []

for index, row in merged.iterrows(): #iterrows changes the format of the dataframe into a table where you can get the values by doing row.mid_h where row corresponds to the row in the original dataframe you want, note it has lower performance as it makes a series for each row
    
    if row.SIGNAL != NONE: #check if current row is a trade
        open_trades_m5.append(TradeM5(row))
        
    for ot in open_trades_m5: #itterate through each still open trade and check that if the current candle we are on leads to a stop loss or take profit, if so the trade is then closed
        ot.update(row)
        if ot.running == False:
            closed_trades_m5.append(ot)
    open_trades_m5 = [x for x in open_trades_m5 if x.running == True] 

    if row.SIGNAL != NONE: #check if current row is a trade
        open_trades_m5.append(TradeM5(row))

In [156]:
df_res_m5 = pd.DataFrame.from_dict([vars(x)for x in closed_trades_m5])

In [157]:
df_res_m5.result.sum()

-144.0